In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/signapoop/Desktop/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer
import networkx as nx
import scipy.sparse as sp

In [3]:
from core.EmbeddingDataSet import EmbeddingDataSet
from core.GraphDataBlock import GraphDataBlock

In [63]:
dataset_name = 'reddit_test'
parent_dir = os.path.abspath('..')
input_dir = os.path.join(parent_dir, 'data')
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(n_batches=1, shuffle=False)
dataset.summarise()

Data blocks of length:  [55334]
Time to create all data (s) = 0.4205
Name of dataset = reddit_test
Input dimension = 602
Number of training samples = 55334
Training labels = True


In [64]:
print(dataset.inputs.shape)
print(len(dataset.labels))
print(dataset.adj_matrix.shape)

(55334, 602)
55334
(55334, 55334)


In [65]:
dataset.inputs

array([[ 2.76121381,  3.51401294, -0.94269743, ..., -0.22735964,
         0.27362608, -0.40100911],
       [-0.24396839, -0.1961025 , -0.05843355, ...,  0.06941389,
         0.24827524,  0.25882556],
       [-0.71172419, -0.58276388, -0.17510535, ..., -0.76102937,
        -0.50193927,  1.39916178],
       ...,
       [-0.52520928, -0.96942525,  0.2018864 , ...,  0.64377226,
         0.13026628,  1.32726385],
       [-0.24396839, -0.35658147, -0.22048894, ...,  0.00778838,
        -0.3666105 ,  0.33776654],
       [-0.3728155 ,  0.19055887, -0.10349861, ..., -0.87750852,
         0.44574836,  0.35144756]])

In [66]:
dataset.adj_matrix

<55334x55334 sparse matrix of type '<class 'numpy.int64'>'
	with 1615610 stored elements in Compressed Sparse Row format>

In [67]:
outfile = 'new_data/{}.npz'.format(dataset_name)
np.savez(outfile, inputs=dataset.inputs, labels=dataset.labels, adj_matrix=dataset.adj_matrix)

In [68]:
npzfile = np.load(outfile)

In [69]:
npzfile.files

['inputs', 'labels', 'adj_matrix']

In [70]:
npzfile['inputs']

array([[ 2.76121381,  3.51401294, -0.94269743, ..., -0.22735964,
         0.27362608, -0.40100911],
       [-0.24396839, -0.1961025 , -0.05843355, ...,  0.06941389,
         0.24827524,  0.25882556],
       [-0.71172419, -0.58276388, -0.17510535, ..., -0.76102937,
        -0.50193927,  1.39916178],
       ...,
       [-0.52520928, -0.96942525,  0.2018864 , ...,  0.64377226,
         0.13026628,  1.32726385],
       [-0.24396839, -0.35658147, -0.22048894, ...,  0.00778838,
        -0.3666105 ,  0.33776654],
       [-0.3728155 ,  0.19055887, -0.10349861, ..., -0.87750852,
         0.44574836,  0.35144756]])

In [75]:
npzfile['adj_matrix'].item()

<55334x55334 sparse matrix of type '<class 'numpy.int64'>'
	with 1615610 stored elements in Compressed Sparse Row format>

In [76]:
class NewEmbeddingDataSet(EmbeddingDataSet):
    """
    Attributes:
        name (str): name of dataset
        data_dir (str): path to dataset folder
        train_dir (str): path to training data file
        test_dir (str): path to test data file
        input_dim (int): number of data features per node
        is_labelled (Boolean): whether underlying class labels are present
        all_data (list[GraphDataBlock]): data inputs packaged into blocks
        all_indices (np.array): input sequence when packaging data into blocks

        inputs (scipy csr matrix): data feature matrix of size n x f
        labels (np.array): data class label matrix of size n x 1
        adj_matrix (scipy csr matrix): adjacency matrix of size n x n
    """

    train_dir = {'cora_test': 'cora_test.npz',
                 'cora_third_test': 'cora_test.npz',
                 'pubmed_test': 'pubmed_test.npz',
                 'citeseer_test': 'citeseer_test.npz',
                 'reddit_full': 'reddit_full.npz',
                 'reddit_test': 'reddit_test.npz',
                 'cora_ml': 'cora_ml.npz',
                 'ms_academic': 'ms_academic.npz'}

    test_dir = train_dir

    def __init__(self, name, data_dir, train=True):
        self.name = name
        self.data_dir = data_dir
        self.train_dir = NewEmbeddingDataSet.train_dir[name]
        self.test_dir = NewEmbeddingDataSet.test_dir[name]
        self.is_labelled = False

        self.all_data = []

        # Extract data from file contents
        data_root = os.path.join(self.data_dir, self.name)
        if train:
            fname = os.path.join(data_root, self.train_dir)
        else:
            assert self.test_dir is not None
            fname = os.path.join(data_root, self.test_dir)
        npzfile = np.load(fname)

        self.inputs = npzfile['inputs']
        self.adj_matrix = npzfile['adj_matrix'].item()
        self.is_labelled = 'labels' in npzfile.files
        if self.is_labelled:
            self.labels = npzfile['labels']

        self.input_dim = self.inputs.shape[1]

        self.all_indices = np.arange(0, self.inputs.shape[0])

        # Convert adj to csr matrix
        self.adj_matrix = sp.csr_matrix(self.adj_matrix)

In [78]:
dataset = NewEmbeddingDataSet('cora_test', input_dir, train=True)
dataset.create_all_data(n_batches=1, shuffle=False)
dataset.summarise()

IndexError: tuple index out of range

In [79]:
dataset.inputs.item()

ValueError: can only convert an array of size 1 to a Python scalar